In [14]:
# load auto reload module
%load_ext autoreload

In [1]:
! python3 --version

Python 3.10.12


In [2]:
import os
import gc

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

import json
import wandb
from tqdm import tqdm

In [3]:
# Print current working directory
# !ls -alh /var/model/Phind-CodeLlama-34B-v2
# Change to /var/model/Phind-CodeLlama-34B-v2
# os.chdir( "/var/model/Phind-CodeLlama-34B-v2" )
# Print current working directory
# os.getcwd()
! ls -alh /var/model/

total 432K
drwxr--r-- 35 1001 1001 4.0K Jan 11 20:33 .
drwxr-xr-x  1 root root 4.0K Jan 17 15:12 ..
-rw-r--r--  1 1001 1001 8.1K Jan  4 22:17 .DS_Store
-rw-r--r--  1 1001 1001 4.0K Jan  4 01:11 ._.DS_Store
drwxr--r--  3 1001 1001 4.0K Aug 22 13:57 .idea
drwxr--r-- 16 1001 1001 4.0K Aug 15 19:52 Auto-GPT
drwxrwxr-x  6 1001 1001 4.0K Nov 19 02:04 CodeLlama-13b-Instruct-hf
drwxrwxr-x  7 1001 1001 4.0K Dec 19 14:10 CodeLlama-7b-Instruct-hf
-rwxr--r--  1 1001 1001 3.3K Feb 28  2023 Dockerfile
drwxrwxr-x 15 1001 1001 4.0K Nov  8 18:06 OpenLLM
drwxrwxr-x  9 1001 1001 4.0K Jan 12 20:09 Phind-CodeLlama-34B-v2
drwxrwxr-x 13 1001 1001 4.0K Dec 13 00:08 TTS
drwxr--r--  6 1001 1001 4.0K Feb 28  2023 ampe
drwxr--r--  5 1001 1001 4.0K Jul  5  2023 cache
-rwxr--r--  1 1001 1001 254K Aug  1 01:17 chat_history.txt
drwxr--r--  5 1001 1001 4.0K Apr 28  2023 coursework
drwxrwxr-x  8 1001 1001 4.0K Nov  3 15:33 cursor-flask-chatbot
drwxr--r--  6 1001 1001 4.0K Sep  3 23:49 cursor-flask-js-websocket
drwxr-xr

In [7]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
%env WANDB_PROJECT=Phind-CodeLlama-34B-v2-peft-fine-tuning

env: WANDB_PROJECT=Phind-CodeLlama-34B-v2-peft-fine-tuning


In [4]:
from xmlschema import XMLSchema

os.chdir( "/var/model/genie-in-the-box/src" )
print( os.getcwd() )
import lib.utils.util         as du
import lib.utils.util_xml     as dux
import lib.utils.util_pytorch as dupt

from lib.agents.agent         import XXX_Agent

from ephemera.prompts.xml_fine_tuning_prompt_generator import XmlFineTuningPromptGenerator


/var/model/genie-in-the-box/src


## Get training dataset

In [16]:
path = "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-train.jsonl"
deepily_dataset_train = du.get_file_as_list( path )[ 0:1000]
deepily_dataset_train = [ json.loads( line ) for line in deepily_dataset_train ]
len( deepily_dataset_train )

1000

In [17]:
path = "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-test.jsonl"
deepily_dataset_test = du.get_file_as_list( path )[ 0:100]
deepily_dataset_test = [ json.loads( line ) for line in deepily_dataset_test ]
# deepily_dataset_test

In [18]:
len( deepily_dataset_test )

100

In [19]:
# Use the Task below and the Input given to write the Response, which is a programmatic instruction that can solve the following Task:
def prompt_instruction_format( sample ):
    
  return f"""### Instruction:
    Use the Task below and the Input given to write a Response that can solve the following Task:

    ### Task:
    {sample['instruction']}

    ### Input:
    {sample['input']}

    ### Response:
    {sample['output']}
    """

In [20]:
for line in prompt_instruction_format( deepily_dataset_test[ 0 ] ).split( "\n" ): print( line )

### Instruction:
    Use the Task below and the Input given to write a Response that can solve the following Task:

    ### Task:
    Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

        You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

        Requirement: You MUST NOT use python code to answer this question.
        Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
        Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

    ### Input:
    
        Below is the raw human voice c

In [21]:
def generate_text( foo_tokenizer, model, question, max_new_tokens=256, debug=False ):
    
    instruction = f"""### Instruction:
    Use the Task below and the Input given to write a Response that can solve the following Task:

    ### Task:
    Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

    You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

    Requirement: You MUST NOT use python code to answer this question.
    Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
    Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

    ### Input:
    
    Below is the raw human voice command transcription formatted using simple XML:
    
    <human>
        <voice-command>{question}</voice-command>
    </human>
    
    The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:
    
    <response>
        <browser-command></browser-command>
        <args></args>
    </response>

    Requirement: The first word of your response MUST be `<response>`

    ### Response:"""
    
    
    device = "cuda:0"
    inputs = foo_tokenizer( instruction, return_tensors="pt" ).to( device )
    
    stop_token_id = foo_tokenizer.encode( "</response>" )[ 0 ]
    
    generation_output = model.generate(
        input_ids=inputs[ "input_ids" ],
        attention_mask=inputs[ "attention_mask" ],
        max_new_tokens=max_new_tokens,
        eos_token_id=stop_token_id,
        pad_token_id=stop_token_id
    )
        
    if debug: 
        print( "generation_output[ 0 ]:", generation_output[ 0 ], end="\n\n" )
        print( "generation_output[ 0 ].shape:", generation_output[ 0 ].shape, end="\n\n" )
    
    # Skip decoding the prompt part of the output   
    input_length = inputs[ "input_ids" ].size( 1 )
    raw_output = foo_tokenizer.decode( generation_output[ 0 ][ input_length: ] )
    
    if debug: 
        print( "raw_output:", raw_output, end="\n\n" )
        print(  "len( raw_output ):", len( raw_output ), end="\n\n")
    
    # response   = raw_output.split( "### Response:" )[ 1 ]
    
    response = raw_output.replace( "</s><s>", "" ).strip()
    
    return response

question = "Ask Google scholar about QLORA and PEFT fine-tuning for XML output, show results in Another tab"

# for line in generate_text( tokenizer, base_model, question ).split( "\n" ): print( line )

In [23]:
import os 
os.chdir( "/var/model/Phind-CodeLlama-34B-v2" )

In [24]:
compute_dtype = getattr( torch, "float16" )

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)
print( bnb_config )
tokenizer              = AutoTokenizer.from_pretrained( "." )
tokenizer.pad_token    = tokenizer.eos_token
tokenizer.padding_side = "right"

# ¡OJO! Why are All the examples I'm finding online turning off the cache here? It makes a huge performance difference: 21 vs 14 tokens per second!
# Now I remember: cashing is suspended while training, should only be used for inference
base_model = AutoModelForCausalLM.from_pretrained(
    ".", quantization_config=bnb_config, device_map="auto", low_cpu_mem_usage=True, use_cache=True, use_flash_attention_2=True
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


BitsAndBytesConfig {
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}


The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [25]:
base_model.hf_device_map

OrderedDict([('model.embed_tokens', 0),
             ('model.layers.0', 0),
             ('model.layers.1', 0),
             ('model.layers.2', 0),
             ('model.layers.3', 0),
             ('model.layers.4', 0),
             ('model.layers.5', 0),
             ('model.layers.6', 0),
             ('model.layers.7', 0),
             ('model.layers.8', 0),
             ('model.layers.9', 0),
             ('model.layers.10', 0),
             ('model.layers.11', 0),
             ('model.layers.12', 0),
             ('model.layers.13', 0),
             ('model.layers.14', 0),
             ('model.layers.15', 0),
             ('model.layers.16', 0),
             ('model.layers.17', 0),
             ('model.layers.18', 0),
             ('model.layers.19', 0),
             ('model.layers.20', 0),
             ('model.layers.21', 1),
             ('model.layers.22', 1),
             ('model.layers.23', 1),
             ('model.layers.24', 1),
             ('model.layers.25', 1),
        

In [23]:
for name, param in base_model.named_parameters():
    print(f"Parameter {name} is on device {param.device}")

Parameter model.embed_tokens.weight is on device cuda:0
Parameter model.layers.0.self_attn.q_proj.weight is on device cuda:0
Parameter model.layers.0.self_attn.k_proj.weight is on device cuda:0
Parameter model.layers.0.self_attn.v_proj.weight is on device cuda:0
Parameter model.layers.0.self_attn.o_proj.weight is on device cuda:0
Parameter model.layers.0.mlp.gate_proj.weight is on device cuda:0
Parameter model.layers.0.mlp.up_proj.weight is on device cuda:0
Parameter model.layers.0.mlp.down_proj.weight is on device cuda:0
Parameter model.layers.0.input_layernorm.weight is on device cuda:0
Parameter model.layers.0.post_attention_layernorm.weight is on device cuda:0
Parameter model.layers.1.self_attn.q_proj.weight is on device cuda:0
Parameter model.layers.1.self_attn.k_proj.weight is on device cuda:0
Parameter model.layers.1.self_attn.v_proj.weight is on device cuda:0
Parameter model.layers.1.self_attn.o_proj.weight is on device cuda:0
Parameter model.layers.1.mlp.gate_proj.weight is on

## Set up training arguments

In [26]:
from peft import LoraConfig, get_peft_config, PeftModel, PeftConfig, get_peft_model, AutoPeftModelForCausalLM

peft_config = LoraConfig(
    r=16, 
    lora_alpha=32, 
    # When target_modules was disabled, it was causing detention layers to be assigned to the CPU, throwing this runtime error:
    # RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! 
    # (when checking argument for argument mat2 in method wrapper_CUDA_mm)
    target_modules=[ "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj" ], 
    lora_dropout=0.10, 
    bias="none", 
    task_type="CAUSAL_LM"
)

In [21]:
del trainingArgs
del trainer
import gc
torch.cuda.empty_cache()
gc.collect()

1878

In [27]:
# Define the training arguments
trainingArgs = TrainingArguments(
    output_dir="./training-results", # Output directory where the model predictions and checkpoints will be stored
    num_train_epochs=4, # Number of training epochs
    per_device_train_batch_size=1, # Batch size per GPU for training
    gradient_accumulation_steps=4,  # Number of update steps to accumulate the gradients for
    gradient_checkpointing=True,# Enable gradient checkpointing
    optim="paged_adamw_32bit", # Optimizer to use
    #save_steps=save_steps,
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    
    # Setting this may help with the warning message: The input hidden states seems to be silently casted in float32, 
    # this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.
    fp16=True,
    # Test to confirm that this works!
    # BTW: according to PHIND, this may actually improve fine-tuning performance as well: https://www.phind.com/search?cache=ygn9dbyl0ij4kotmgns2nsrw
    
    bf16=False,
    # tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    #max_steps=max_steps,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    report_to="wandb",
    seed=42
)
# Create the trainer
trainer = SFTTrainer(
    model=base_model,
    train_dataset=deepily_dataset_train,
    eval_dataset=deepily_dataset_test,
    peft_config=peft_config,
    max_seq_length=4096, #2048,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [28]:
def print_trainable_parameters( model ):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [29]:
print_trainable_parameters( base_model )

trainable params: 108,920,832 || all params: 17,243,447,296 || trainable%: 0.63


In [30]:
for name, param in base_model.named_parameters():
    print(f"Parameter {name} is on {param.device}")


Parameter model.embed_tokens.weight is on cuda:0
Parameter model.layers.0.self_attn.q_proj.base_layer.weight is on cuda:0
Parameter model.layers.0.self_attn.q_proj.lora_A.default.weight is on cuda:0
Parameter model.layers.0.self_attn.q_proj.lora_B.default.weight is on cuda:0
Parameter model.layers.0.self_attn.k_proj.base_layer.weight is on cuda:0
Parameter model.layers.0.self_attn.k_proj.lora_A.default.weight is on cuda:0
Parameter model.layers.0.self_attn.k_proj.lora_B.default.weight is on cuda:0
Parameter model.layers.0.self_attn.v_proj.base_layer.weight is on cuda:0
Parameter model.layers.0.self_attn.v_proj.lora_A.default.weight is on cuda:0
Parameter model.layers.0.self_attn.v_proj.lora_B.default.weight is on cuda:0
Parameter model.layers.0.self_attn.o_proj.base_layer.weight is on cuda:0
Parameter model.layers.0.self_attn.o_proj.lora_A.default.weight is on cuda:0
Parameter model.layers.0.self_attn.o_proj.lora_B.default.weight is on cuda:0
Parameter model.layers.0.mlp.gate_proj.base

In [31]:
trainer.train()

#stop reporting to wandb
wandb.finish()

# save model
trainer.save_model()

print( "Model saved" )

# {'loss': 0.5582, 'learning_rate': 1.1111111111111112e-05, 'epoch': 0.03}
# {'loss': 0.512, 'learning_rate': 2.2222222222222223e-05, 'epoch': 0.05}
# {'loss': 0.4509, 'learning_rate': 3.3333333333333335e-05, 'epoch': 0.07}
# {'loss': 0.3753, 'learning_rate': 4.4444444444444447e-05, 'epoch': 1.0}
# {'loss': 0.3123, 'learning_rate': 5.555555555555556e-05, 'epoch': 1.03}
# {'loss': 0.259, 'learning_rate': 6.666666666666667e-05, 'epoch': 1.05}
# {'loss': 0.1952, 'learning_rate': 7.777777777777778e-05, 'epoch': 1.08}
# {'loss': 0.1454, 'learning_rate': 8.888888888888889e-05, 'epoch': 2.01}
# {'loss': 0.1129, 'learning_rate': 0.0001, 'epoch': 2.03}
# {'loss': 0.0995, 'learning_rate': 0.00011111111111111112, 'epoch': 2.06}
# {'loss': 0.0877, 'learning_rate': 0.00012222222222222224, 'epoch': 2.08}
# {'loss': 0.0815, 'learning_rate': 0.00013333333333333334, 'epoch': 3.01}

wandb: Currently logged in as: ricardo-felipe-ruiz. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.


{'loss': 0.4885, 'learning_rate': 0.00019977186146800707, 'epoch': 0.21}
{'loss': 0.2683, 'learning_rate': 0.00019721724257579907, 'epoch': 0.41}
{'loss': 0.1532, 'learning_rate': 0.00019189578116202307, 'epoch': 0.62}
{'loss': 0.1068, 'learning_rate': 0.00018395892819696389, 'epoch': 0.82}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0894, 'learning_rate': 0.00017363256976511972, 'epoch': 1.03}
{'loss': 0.0803, 'learning_rate': 0.0001612105982547663, 'epoch': 1.24}
{'loss': 0.0745, 'learning_rate': 0.0001470465480602756, 'epoch': 1.44}
{'loss': 0.069, 'learning_rate': 0.00013154353384852558, 'epoch': 1.65}
{'loss': 0.0651, 'learning_rate': 0.00011514277775045768, 'epoch': 1.86}


...

{'loss': 0.0619, 'learning_rate': 9.83110519986069e-05, 'epoch': 2.06}
{'loss': 0.0505, 'learning_rate': 8.15273943982811e-05, 'epoch': 2.27}
{'loss': 0.0488, 'learning_rate': 6.526947471551798e-05, 'epoch': 2.47}
{'loss': 0.0478, 'learning_rate': 5.000000000000002e-05, 'epoch': 2.68}
{'loss': 0.0489, 'learning_rate': 3.615354575300166e-05, 'epoch': 2.89}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0464, 'learning_rate': 2.4124187730720917e-05, 'epoch': 3.09}
{'loss': 0.0409, 'learning_rate': 1.425428638693489e-05, 'epoch': 3.3}
{'loss': 0.0401, 'learning_rate': 6.824743154333157e-06, 'epoch': 3.51}
{'loss': 0.0401, 'learning_rate': 2.0470058747505516e-06, 'epoch': 3.71}
{'loss': 0.0383, 'learning_rate': 5.705090702819993e-08, 'epoch': 3.92}
{'train_runtime': 3831.7001, 'train_samples_per_second': 0.101, 'train_steps_per_second': 0.025, 'train_loss': 0.09723736295321335, 'epoch': 3.96}


train/epoch,▁▁▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇███
train/learning_rate,███▇▇▇▆▆▅▄▄▃▃▂▂▁▁▁▁
train/loss,█▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,3.96
train/global_step,96


Model saved


In [19]:
# wandb.finish()

In [28]:
for line in generate_text( tokenizer, base_model, question ).split( "\n" ): print( line )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


generation_output[ 0 ]: tensor([    1,   835,  2799,  4080, 29901,    13,  1678,  4803,   278,  9330,
         2400,   322,   278, 10567,  2183,   304,  2436,   263, 13291,   393,
          508,  4505,   278,  1494,  9330, 29901,    13,    13,  1678,   835,
         9330, 29901,    13,  1678,  3575,  4982,   338,   304,  2313,   824,
          278,  7609,   310,   263,  5199,  7314,  1899,  1301,  3395,   322,
        14240,   372,   964,   263,  3918,  1891,  1899,   393,   263,  4714,
          373,   596,  6601,   723,  2274, 29889,    13,    13,  1678,   887,
          674,   367,  2183,   263,  5199,  7314,  1899,   322,   263,  1051,
          310,  1950,  3918,  1891,  8260, 29889,   887,  1818,  6755,   278,
         1959,  3918,  1891,  1899,   515,   278,  1494,  1051, 29901, 16218,
         4478,   716,  4434,   742,   525,  4478,  1857,  4434,   742,   525,
         4478,  5386,   716,  4434,   742,   525,  4478,  5386,  1857,  4434,
          742,   525,  4478,  5386, 2134

In [47]:
# Drops 16.4/19.0 GB per GPU down to 3.25 GB per GPU!
import gc
base_model = None 
adapter_plus_model = None
torch.cuda.empty_cache() 
gc.collect()

493

In [6]:
import torch
getattr( torch, "float16" )

torch.float16

In [7]:
os.chdir( "/var/model/Phind-CodeLlama-34B-v2" )
os.getcwd()

'/var/model/Phind-CodeLlama-34B-v2'

In [8]:
import torch

compute_dtype = getattr( torch, "float16" )

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)
print( bnb_config )
tokenizer              = AutoTokenizer.from_pretrained( "." )
tokenizer.pad_token    = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    ".", quantization_config=bnb_config, device_map="auto", low_cpu_mem_usage=True, use_cache=True, use_flash_attention_2=True
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


BitsAndBytesConfig {
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}


The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
# dupt.print_device_allocation( base_model )


In [9]:
from peft import PeftModel
adapter_plus_model = PeftModel.from_pretrained( base_model, "adapters/01-browser-vox-command", use_flash_attention_2=True )

In [17]:
# from accelerate import Accelerator
# 
# accelerator = Accelerator()
# 
# adapter_plus_model = accelerator.prepare( adapter_plus_model )


In [21]:
dupt.print_device_allocation( adapter_plus_model )

base_model.model.model.embed_tokens.weight: cuda:0
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight: cuda:0
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight: cuda:0
base_model.model.

In [36]:
import pandas as pd

In [28]:
test_df = pd.read_json( "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-validate.jsonl", lines=True ).sample( 100, random_state=42 )
test_df.shape

(100, 5)

In [29]:
%autoreload

xml_ftp_generator = XmlFineTuningPromptGenerator( path_prefix="/var/model/genie-in-the-box", debug=True, verbose=False )

test_df = xml_ftp_generator.generate_responses( test_df, tokenizer=tokenizer, model=adapter_plus_model, switch="huggingface", model_name=XXX_Agent.PHIND_34B_v2 )
test_df = xml_ftp_generator.validate_responses( test_df )

Commands file for [search new tab] exists: True
Commands file for [search current tab] exists: True
Commands file for [search google new tab] exists: True
Commands file for [search google current tab] exists: True
Commands file for [search google scholar new tab] exists: True
Commands file for [search google scholar current tab] exists: True

Using HuggingFace model_name [TGI/Phind-CodeLlama-34B-v2] in memory...

Processing call [001] out of [100] = [1.0%]... 
Asking LLM [TGI/Phind-CodeLlama-34B-v2]...
Asking LLM [TGI/Phind-CodeLlama-34B-v2]... Done! in 4,585 ms
Tokens per second [36.2]
Response: [<response><browser-command>search google scholar current tab</browser-command><args>URLError</args></response>]

Processing call [002] out of [100] = [2.0%]... 
Asking LLM [TGI/Phind-CodeLlama-34B-v2]...
Asking LLM [TGI/Phind-CodeLlama-34B-v2]... Done! in 5,577 ms
Tokens per second [40.5]
Response: [<response><browser-command>search google scholar new tab</browser-command><args>How can system

In [37]:
xml_ftp_generator.print_validation_stats( test_df )

# ------------------------------------------------------------------------------------------------------------------------
# - Validation Stats w/ adapter loaded on top
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 100.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 99.0%
# Response has correct values 99.0%
#  Browser command is correct 99.0%
#             Args is correct 100.0%